<a href="https://colab.research.google.com/github/amasl0/myKNN/blob/main/VK_friends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import libraries

In [2]:
import pandas as pd
import numpy as np
import json
import requests
from tqdm import tqdm
import os 
import networkx as nx
import csv

### Берём у группы их vk_id

In [ ]:
json_dict = json.loads('["h ttps://vk.com/amaslo0","https://vk.com/dmitriy_mishka","https://vk.com/zvfrc","https://vk.com/ruzakovskiy","https://vk.com/shis_213","https://vk.com/space1engine","https://vk.com/zannazz","https://vk.com/spymsx1337","https://vk.com/ja2018ck","https://vk.com/m_nkfrv","https://vk.com/renchiqq","https://vk.com/0x51c","https://vk.com/ricktusempra","https://vk.com/nikita.starling","https://vk.com/vladtopchu","https://vk.com/weaklyzxc","https://vk.com/disdinortimy028484892","https://vk.com/gendorosan","https://vk.com/id323208790","https://vk.com/id173792332","https://vk.com/semendyaevv","https://vk.com/nikitagabov","https://vk.com/i_need_more_dragons","https://vk.com/id338686058","https://vk.com/shoh0107"]') 

In [ ]:
list_students = []

with open('Students.txt', 'r') as f:
    text = f.read()
    text = text.replace(' ', '').split('\n')
    list_students = [ [text[ix],text[ix+1], text[ix+2], text[ix+3]] for ix in range(0,len(text), 4)]

In [ ]:
list_students

[['https://vk.com/amaslo0', '181948702', 'Андрей', 'Маслов'],
 ['https://vk.com/dmitriy_mishka', '154308430', 'Дмитрий', 'Медведев'],
 ['https://vk.com/zvfrc', '152087036', 'Максим', 'Зуенко'],
 ['https://vk.com/ruzakovskiy', '143779216', 'Егор', 'Рузайкин'],
 ['https://vk.com/shis_213', '185215592', 'Ильяс', 'Шакиров'],
 ['https://vk.com/space1engine', '161594007', 'Максим', 'Михеев'],
 ['https://vk.com/zannazz', '472583599', 'Анна', 'Зайцева'],
 ['https://vk.com/spymsx1337', '113001678', 'Максим', 'Марголин'],
 ['https://vk.com/ja2018ck', '183648122', 'Евгений', 'Малиновский'],
 ['https://vk.com/m_nkfrv', '196404046', 'Михаил', 'Никифоров'],
 ['https://vk.com/renchiqq', '85770676', 'Ринат', 'Галямшин'],
 ['https://vk.com/0x51c', '355133495', 'Влад', 'Минаев'],
 ['https://vk.com/ricktusempra', '137872152', 'Андрей', 'Форшаков'],
 ['https://vk.com/nikita.starling', '102711908', 'Никита', 'Скворцов'],
 ['https://vk.com/vladtopchu', '126568936', 'Влад', 'Топчу'],
 ['https://vk.com/weakly

In [ ]:
df_group_students = pd.DataFrame()

In [ ]:
df_group_students = df_group_students.append( list_students, ignore_index=True )

In [ ]:
df_group_students = df_group_students.rename(columns={ 0:'href', 1:'id', 2:'fname', 3:'lname'})

### Берём друзей студентов из ВК

In [ ]:
with open ('untitled.txt', 'r') as f:
    token = str(f.readline())
    
def getFriend (token, user_id, level_friend):
    list_data = []
    try:
        list_friend = requests.get("https://api.vk.com/method/friends.get",
                        params={
                            "user_id": user_id,
                            "order": "name",
                            "count": 5000,
                            "offset": 0,
                             "fields": "nickname",
                            "access_token": token,
                            "v": 5.126,
                        }).json()

        if not "error" in list_friend.keys():
            if list_friend["response"]['items'] != None:
                list_friend = list_friend['response']['items']
                for friend in list_friend:
                    if not "deactivated" in friend and friend["is_closed"] != True:
                        list_data.append([ user_id, level_friend, friend['id'], friend['first_name'], friend['last_name'] ])
    except:
        return list_data
    return list_data


In [ ]:
import time

al_dic = []

## итерируемся по группе
for tup in df_group_students.itertuples():
    res = getFriend(token, tup.id, 1)
    if type(res) == list and res != None:
        al_dic.extend(res)
    time.sleep(0.05)

In [ ]:
df_friends = pd.DataFrame(al_dic,columns=['main_id','level_friend','id_friend','first_name','last_name'])

In [ ]:
df_friends

,main_id,level_friend,id_friend,first_name,last_name
0,181948702,1,442387982,ピンクの雨,カップ・カップ
1,181948702,1,581054682,叫张,志
2,181948702,1,194384429,Alexander,Gerasimov
3,181948702,1,430679842,Alexander,Rykalin
4,181948702,1,163473645,Alexander,Tsvetkov
...,...,...,...,...,...
2985,312616596,1,358526702,Valya,Novgorodova
2986,312616596,1,326542225,Vika,Volkanitina
2987,312616596,1,72207822,Viktor,Melnikov
2988,312616596,1,474174041,Yury,Nano


In [ ]:
al_dic = []
df_second_friends = pd.DataFrame()

counter = 0

for tup in tqdm(df_friends.itertuples(), total=df_friends.shape[0]):
    res = getFriend(token, tup.id_friend, 2)
    if type(res) == list and res != None:
        al_dic.extend(res)
    time.sleep(0.05)
    
    if len(al_dic) > 100000:
        pd.DataFrame(al_dic, columns=['main_id','level_friend','id_friend','first_name','last_name']).to_csv(f'parts/part-{counter}.csv', index=False)
        counter += 1
        al_dic.clear()

100%|██████████| 2990/2990 [24:05<00:00,  2.07it/s] 


In [ ]:
pd.DataFrame(al_dic, columns=['main_id','level_friend','id_friend','first_name','last_name']).to_csv(f'parts/part-{counter}.csv', index=False)

In [ ]:
counter

24

### Объединяем все части в один dataframe

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_friends = pd.DataFrame(columns=['main_id','level_friend','id_friend','first_name','last_name'])

In [4]:
CURPATH = 'drive/MyDrive/parts'

for part in os.listdir(CURPATH):
    df_friends = df_friends.append( pd.read_csv( '/'.join(( CURPATH, part )) ) )

In [5]:
len(df_friends)

2019126

In [6]:
df_friends = df_friends.drop_duplicates()

In [7]:
df_friends

,main_id,level_friend,id_friend,first_name,last_name
0,442387982,2,413144204,Alexandra,Zavodnova
1,442387982,2,181948702,Andrey,Maslov
2,442387982,2,271636767,Dmitry,Elesichev
3,442387982,2,246816062,Germiona,Orlova
4,442387982,2,440976960,Pangeleya,Atamankina
...,...,...,...,...,...
2985,312616596,1,358526702,Valya,Novgorodova
2986,312616596,1,326542225,Vika,Volkanitina
2987,312616596,1,72207822,Viktor,Melnikov
2988,312616596,1,474174041,Yury,Nano


### Делаем граф в networkx

In [8]:
G = nx.Graph()

In [11]:
### нужно узнать id уникальных пользователей
## добавляем вершины графа
df_friends_short = df_friends.loc[ df_friends['level_friend'] == 1]

G.add_nodes_from(set( df_friends_short['main_id'].unique()) | set(df_friends_short['id_friend'].unique()))

In [12]:
## добавляем ребра
G.add_edges_from(df_friends_short[['main_id','id_friend']].values.tolist())

### Степень центральности

In [13]:
deg_centrality = nx.degree_centrality(G)

In [36]:
def getPeople ( centrality, count=5 ):
    res_centrality = sorted(centrality.items(), key=lambda item: item[1], reverse=True)
    return df_friends.loc[
            df_friends.id_friend.isin(
                    (item[0] for item in res_centrality[:count])
                ), ['first_name', 'last_name']
         ].drop_duplicates()

In [37]:
getPeople(deg_centrality)

,first_name,last_name
2486,Gennady,Skakodub
12059,Egor,Ruzaikin
14733,Nikita,Skvortsov
19859,Evgeny,Malinovsky
38931,Mikhail,Nikiforov


### Центральность по посредничеству

In [38]:
bet_centrality = nx.betweenness_centrality(G, normalized = True, 
                                              endpoints = False)

In [39]:
getPeople(bet_centrality)

,first_name,last_name
1,Andrey,Maslov
2486,Gennady,Skakodub
2488,Igor,Kotilevets
19859,Evgeny,Malinovsky
38931,Mikhail,Nikiforov


### Центральность по близости



In [40]:
closeness_centrality = nx.closeness_centrality(G)
getPeople(closeness_centrality)

,first_name,last_name
2486,Gennady,Skakodub
2488,Igor,Kotilevets
2495,Leonid,Saburov
19859,Evgeny,Malinovsky
38931,Mikhail,Nikiforov


### Центральность по значению вектора



In [41]:
eigenvector_centrality = nx.eigenvector_centrality_numpy(G)
getPeople(eigenvector_centrality)

,first_name,last_name
2488,Igor,Kotilevets
2495,Leonid,Saburov
19859,Evgeny,Malinovsky
20654,Timofey,Shafranov
38931,Mikhail,Nikiforov
